# Simple CNN notebook

## Import Libraries

In [1]:
%matplotlib inline
import numpy as np
from utils.helper import read_data, blur, to_real, morphology
import cv2
import matplotlib.pyplot as plt
from tensorflow import keras
FOLDER = "data/data_big/"

## Load data

In [2]:
data_empty = read_data(FOLDER + "empty.csv")
data_1p = read_data(FOLDER + "1p.csv")
data_2p = read_data(FOLDER + "2p.csv")
data_3p = read_data(FOLDER + "3p.csv")
print("----------------Shapes----------------")
print("empty:", data_empty.shape)
print("1p:", data_1p.shape)
print("2p:", data_2p.shape)
print("3p:", data_3p.shape)

----------------Shapes----------------
empty: (480, 5, 3, 64, 64)
1p: (480, 5, 3, 64, 64)
2p: (480, 5, 3, 64, 64)
3p: (480, 5, 3, 64, 64)


In [3]:
LENGTH_EMPTY = data_empty.shape[0]
LENGTH_1P = data_1p.shape[0]
LENGTH_2P = data_2p.shape[0]
LENGTH_3P = data_3p.shape[0]

In [4]:
data_empty_summed = to_real(np.sum(data_empty, axis=1)) 
data_1p_summed = to_real(np.sum(data_1p, axis=1))
data_2p_summed = to_real(np.sum(data_2p, axis=1))
data_3p_summed = to_real(np.sum(data_3p, axis=1))

/Users/kostiantynlavronenko/Desktop/Eestech/hackathon-milan/submission/code/utils/helper.py:41: ComplexWarning: Casting complex values to real discards the imaginary part
  return data_real.astype(float)


In [5]:
data_0 = np.moveaxis(data_empty_summed, 1,3)
data_1 = np.moveaxis(data_1p_summed, 1,3)
data_2 = np.moveaxis(data_2p_summed, 1,3)
data_3 = np.moveaxis(data_3p_summed, 1,3)
print(data_0.shape)
print(data_1.shape)
print(data_2.shape)
print(data_3.shape)

(480, 64, 64, 3)
(480, 64, 64, 3)
(480, 64, 64, 3)
(480, 64, 64, 3)


In [6]:
data_tot = np.concatenate((data_0, data_1, data_2, data_3), axis=0)
data_tot.shape

(1920, 64, 64, 3)

### Create Labels

In [7]:
labels_empty = np.zeros(LENGTH_EMPTY)
labels_1p = np.ones(LENGTH_1P)
labels_2p = np.ones(LENGTH_2P) * 2
labels_3p = np.ones(LENGTH_3P) * 3
labels = np.concatenate((labels_empty, labels_1p, labels_2p, labels_3p), axis=0)
print("Labels shape:", labels.shape)

Labels shape: (1920,)


In [8]:
train_labels_one_hot = np.eye(4)[labels.astype(int)]
print("Labels abels one hot shape:", train_labels_one_hot.shape)

Labels abels one hot shape: (1920, 4)


In [9]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data_tot, train_labels_one_hot, test_size=0.1)
print(f"Shape train_X: {train_X.shape}")
print(f"Shape test_X: {test_X.shape}")
print(f"Shape train_y: {train_y.shape}")
print(f"Shape test_y: {test_y.shape}")

Shape train_X: (1728, 64, 64, 3)
Shape test_X: (192, 64, 64, 3)
Shape train_y: (1728, 4)
Shape test_y: (192, 4)


In [17]:
models = keras.models
layers = keras.layers

model = models.Sequential(
    [
        layers.Conv2D(8,
                      input_shape = (64,64,3),
                      kernel_size = 5,
                      padding = "same",
                      activation = "relu"),
        layers.MaxPooling2D(2,
                            strides = None,
                            padding = "valid"),
        layers.Conv2D(8,
                      input_shape = (32,32,3),
                      kernel_size = 3,
                      padding = "same",
                      activation = "relu"),
        layers.MaxPooling2D(2,
                    strides = None,
                    padding = "valid"),
        layers.Flatten(),
        layers.Dense(4),
        layers.Activation("softmax")
    ]
)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 64, 64, 8)         608       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 8)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 8)         584       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 8)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 4)                

In [18]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)

model.fit(
    train_X, train_y,
    epochs  = 10,
    verbose = 1,
    validation_split = 0.1,
    shuffle= True,
    callbacks=keras.callbacks.EarlyStopping(patience = 10)
)

model.save('models/CNN')

Epoch 1/10
49/49 [==============================] - 2s 35ms/step - loss: 1.3987 - accuracy: 0.5704 - val_loss: 0.4742 - val_accuracy: 0.8555
Epoch 2/10
49/49 [==============================] - 1s 28ms/step - loss: 0.3710 - accuracy: 0.8765 - val_loss: 0.2804 - val_accuracy: 0.9191
Epoch 3/10
49/49 [==============================] - 2s 31ms/step - loss: 0.2548 - accuracy: 0.9125 - val_loss: 0.2360 - val_accuracy: 0.8786
Epoch 4/10
49/49 [==============================] - 1s 30ms/step - loss: 0.1893 - accuracy: 0.9441 - val_loss: 0.2583 - val_accuracy: 0.8844
Epoch 5/10
49/49 [==============================] - 2s 31ms/step - loss: 0.1443 - accuracy: 0.9550 - val_loss: 0.1876 - val_accuracy: 0.9306
Epoch 6/10
49/49 [==============================] - 1s 31ms/step - loss: 0.1046 - accuracy: 0.9698 - val_loss: 0.1136 - val_accuracy: 0.9827
Epoch 7/10
49/49 [==============================] - 2s 31ms/step - loss: 0.0834 - accuracy: 0.9820 - val_loss: 0.1061 - val_accuracy: 0.9653
Epoch 8/10
49

INFO:tensorflow:Assets written to: models/CNN/assets


INFO:tensorflow:Assets written to: models/CNN/assets


In [19]:
loss, aacuracy = model.evaluate(test_X, test_y)
print(f"Loss: {loss}")
print(f"Accuracy: {aacuracy}")

6/6 [==============================] - 0s 11ms/step - loss: 0.0885 - accuracy: 0.9635
Loss: 0.08851626515388489
Accuracy: 0.9635416865348816
